In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from torch.utils.data import DataLoader, Dataset, random_split
from lib.checkpoint import load_checkpoint, save_checkpoint
import torch.optim as optim
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np

import time

In [2]:
LSTM_NEUROES = 13*13*13


In [3]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            return False

        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            return False
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
            return False


In [4]:

class LSTM(nn.Module):
    def __init__(self, input_size=LSTM_NEUROES, hidden_size=128, num_layers=2, output_size=LSTM_NEUROES):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.input_fc = nn.Linear(input_size + output_size, input_size)  # 映射到 input_size

    def forward(self, input_t, prev_output, h_0, c_0):
        if prev_output is not None:
            input_t = torch.cat((input_t, prev_output), dim=1)
            input_t = self.input_fc(input_t)  # 映射到原始input_size

        input_t = input_t.unsqueeze(1)  # 调整形状为 (batch_size, seq_len, input_size)
        out, (h_0, c_0) = self.lstm(input_t, (h_0, c_0))
        out = self.fc(out[:, -1, :])
        return out, h_0, c_0


In [5]:
class CTNN3DDecoder(nn.Module):
    def __init__(self):
        super(CTNN3DDecoder, self).__init__()
        self.fc = nn.Linear(LSTM_NEUROES, 8*8*8*16)
        
        self.cov1 = nn.ConvTranspose3d(in_channels = 16, out_channels = 14,
                                kernel_size = 3, stride = 2, padding = 2)
        self.cov2 = nn.ConvTranspose3d(in_channels = 14, out_channels = 12,
                                kernel_size = 3, stride = 2, padding = 2)
        self.relu1 = nn.ReLU()
        
        self.cov3 = nn.ConvTranspose3d(in_channels = 12, out_channels = 10,
                                kernel_size = 3, stride = 2, padding = 2)
        self.cov4 = nn.ConvTranspose3d(in_channels = 10, out_channels = 8,
                                kernel_size = 3, stride = 2, padding = 2)
        self.relu2 = nn.ReLU()
                
        self.cov5 = nn.ConvTranspose3d(in_channels = 8, out_channels = 6,
                                kernel_size = 3, stride = 2, padding = 2)
        self.cov6 = nn.ConvTranspose3d(in_channels = 6, out_channels = 4,
                                kernel_size = 3, stride = 2, padding = 2)
        self.relu3 = nn.ReLU()
        self.upsample = nn.Upsample(size=(32, 32, 32), mode='nearest')        
        
        # self.cov4 = nn.Conv3d(in_channels = 40, out_channel = 1, )
        
    def forward(self, x):
        out = self.fc(x)
        out = out.view(1, 16, 8, 8, 8)
        print(out.shape)
        
        out = self.cov1(out)
        out = self.cov2(out)
        out = self.relu1(out)
        print(out.shape)

        out = self.cov3(out)
        out = self.cov4(out)
        out = self.relu2(out)
        print(out.shape)
        
        out = self.cov5(out)
        out = self.cov6(out)
        out = self.relu3(out)
        print(out.shape)
        out = self.upsample(out)
        print(out.shape)
        
model = CTNN3DDecoder()
test = torch.randn(LSTM_NEUROES)
model(test)


torch.Size([1, 16, 8, 8, 8])
torch.Size([1, 12, 23, 23, 23])
torch.Size([1, 8, 83, 83, 83])
torch.Size([1, 4, 323, 323, 323])
torch.Size([1, 4, 32, 32, 32])


In [6]:
class LSTMDecoder(nn.Module):
    def __init__(self):
        super(LSTMDecoder, self).__init__()
        self.lstm = LSTM()
        self.decoder = CTNN3DDecoder()
        
    def forward(self, x, prev_output, h_0, c_0):
        out, h_0, c_0 = self.lstm(x, prev_output, h_0, c_0)
        out = self.decoder(out)
        return out, h_0, c_0


In [7]:
class dataset(Dataset):
    def __init__(self, datas, voxel):
        self.datas = datas
        self.voxel = voxel

    def __len__(self):
        return len(self.datas)

    def __getitem__(self, index):
        return self.datas[index], self.voxel[index]

In [8]:
def train_subdata(epoch, datas, model, criterion, optimizer, device):
    data_len = len(datas)
    train_data, val_data = random_split(datas, [int(data_len*0.8), data_len-int(data_len*0.8)])
    print("Epoch:{} Train Size:{} Val Size:{}".format(epoch, len(train_data), len(val_data)))
    
    train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=1, shuffle=True)
    train_logs = []
    val_logs = []
    # move data and model to device
    model.to(device)
    val_data = val_data.to(device)
    train_data = train_data.to(device)
    criterion.to(device)
    
    start= time.time()
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()

        batch_size = inputs.size(0)
        h_0 = torch.zeros(model.num_layers, batch_size, model.hidden_size).to(inputs.device)
        c_0 = torch.zeros(model.num_layers, batch_size, model.hidden_size).to(inputs.device)
        prev_output = None

        for t in range(seq_len):
            input_t = inputs[:, t, :]
            output, h_0, c_0 = model(input_t, prev_output, h_0, c_0)
            prev_output = output

        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        train_logs.append(loss.item())
        
    model.eval()
    with torch.no_grad():
        for inputs, targets in val_loader:
            batch_size = inputs.size(0)
            h_0 = torch.zeros(model.num_layers, batch_size, model.hidden_size).to(inputs.device)
            c_0 = torch.zeros(model.num_layers, batch_size, model.hidden_size).to(inputs.device)
            prev_output = None

            for t in range(seq_len):
                input_t = inputs[:, t, :]
                output, h_0, c_0 = model(input_t, prev_output, h_0, c_0)
                prev_output = output

            loss = criterion(output, targets)
            val_logs.append(loss.item())
    end = time.time()
    
    print("Epoch:{} Sub Train Time:{:.2f} Train Loss:{:.4f} Val Loss:{:.4f}".format(epoch, end-start, sum(train_logs)/len(train_logs), sum(val_logs)/len(val_logs)))
    return sum(train_logs)/len(train_logs), sum(val_logs)/len(val_logs)

In [9]:
def run_training(file_path, device, checkpoint_path):
    model = LSTMDecoder()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    early_stopping = EarlyStopping(patience=5, min_delta=0.0001)
    epoch_loss = []
    num_epochs = 20
    
    start_epoch, epoch_losses, last_file, train_loss, val_loss = load_checkpoint(checkpoint_path, model, optimizer, device)



    model.train()
    for epoch in range(num_epochs):
        start = time.time()
        cnt = 0
        train_loss = []
        val_loss = []
        datas = []
        resume = (last_file is not None)
        skip_cnt = 0
        for root, dirs, files in os.walk(file_path):
            start_io = time.time()
            for file in files:
                if resume:
                    if last_file == os.path.join(root, file):
                        print('Resuming from {}, Skip {} Files'.format(last_file, skip_cnt))
                        resume = False
                    skip_cnt += 1
                    continue
                
                file_name = os.path.join(root, file)
                

In [10]:
# 初始化模型
model = LSTM()
batch_size = 1

# 初始化隐藏层状态
h_0 = torch.zeros(model.num_layers, batch_size, model.hidden_size)
c_0 = torch.zeros(model.num_layers, batch_size, model.hidden_size)

# 第一次输入
input_1 = torch.randn(batch_size, 1, LSTM_NEUROES)  # (batch_size, seq_len, input_size)
output, h_n, c_n = model(input_1, h_0, c_0)
print("第一次输出:", output)

# 第二次输入，使用第一次的输出作为一部分输入
input_2 = torch.randn(batch_size, 1, LSTM_NEUROES)  # 另外的输入
new_input = torch.cat((input_2, output.unsqueeze(1)), dim=2)  # 合并输出作为输入的一部分
new_input = model.input_fc(new_input)  # 映射到原始input_size

output, h_n, c_n = model(new_input, h_n, c_n)
print("第二次输出:", output)

# 多次迭代输入进行修正
num_iterations = 5
for i in range(num_iterations):
    input_next = torch.randn(batch_size, 1, LSTM_NEUROES)  # 另外的输入
    new_input = torch.cat((input_next, output.unsqueeze(1)), dim=2)  # 合并输出作为输入的一部分
    new_input = model.input_fc(new_input)  # 映射到原始input_size

    output, h_n, c_n = model(new_input, h_n, c_n)
    print(f"第{i+3}次输出:", output)

TypeError: forward() missing 1 required positional argument: 'c_0'